In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import warnings
import time

warnings.filterwarnings("ignore")

url_tkey = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/list"
url_tdetail = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/get-details"
headers = {
	"X-RapidAPI-Key": "e958b02818msh7e68a4278458a41p10d111jsn2ea01e805185", #change API Key here
	"X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
}
import warnings
warnings.filterwarnings("ignore")

In [6]:
stocks = ['GME',
 'TSLA',
 'AMD',
 'NVDA',
 'BBBY',
 'AAPL',
 'MSFT',
 'AMC',
 'AMZN',
 'F',
 'PLTR',
 'WISH',
 'NFLX',
 'TA',
 'ATVI',
 'PTON',
 'PYPL',
 'COIN',
 'DKNG',
 'INTC',
 'META',
 'RIVN',
 'CLOV',
 'GOOGL',
 'VC',
 'Z',
 'BB',
 'CRSP',
 'MRNA',
 'JD',
 'SOFI',
 'ASML',
 'RBLX',
 'ZM',
 'EA',
 'LCID',
 'FUBO',
 'TTD',
 'NKLA',
 'WBD',
 'CD',
 'COST',
 'MNMD',
 'BASE',
 'NOK',
 'RICK',
 'FANG',
 'AFRM',
 'EVER',
 'FREE',
 'TTWO',
 'TTCF',
 'BYND',
 'PLUS',
 'BLUE',
 'ALLO',
 'DISH',
 'DOCU',
 'WW',
 'HEAR',
 'MVIS',
 'EVGO',
 'UNIT',
 'CLNE',
 'SJ',
 'FCEL',
 'FAT',
 'APP',
 'ROKU',
 'PLUG',
 'NEXT',
 'SAVA',
 'HST',
 'AOSL',
 'MELI',
 'CRSR',
 'GFS',
 'PARA',
 'TECH',
 'NICE',
 'QCOM',
 'ABNB',
 'PDD',
 'SLDP',
 'KIDS',
 'UPST',
 'CSX',
 'DISA',
 'CAMT',
 'PTC',
 'PTRA',
 'ROIC',
 'HOWL',
 'BLNK',
 'AVPT',
 'TIL',
 'AMAT',
 'LYFT',
 'CSCO',
 'PEP'] #initialise list of stocks

In [5]:
stocks = ["AAPL"] # running 1 stock here as example, as the api has a limit of 500 calls per month

In [8]:
def retrieve_earningscall(stock, until = 0, since = 0):
	stock_list = []
	id_dict = {}
	content_list = []
	title_list = []
	date_list = []
	querystring_tkey = {"id":stock,"until":until,"since":since,"size":"40","number":"1"}
	response = requests.request("GET", url_tkey, headers=headers, params=querystring_tkey)
	for d in json.loads(response.text)["data"]:
		id_dict[d["id"]] = d["attributes"]["title"]
	for id, title in id_dict.items():
		if ("Earnings Call Transcript" in title) or ("Earning Call Transcript" in title):
			try:
				querystring_tdetail = {"id":id}
				response = requests.request("GET", url_tdetail, headers=headers, params=querystring_tdetail)
				title = json.loads(response.text)["data"]["attributes"]["title"]
				stock_list.append(stock)
				content_list.append(json.loads(response.text)["data"]["attributes"]["content"])
				title_list.append(title)
				date_list.append(json.loads(response.text)["data"]["attributes"]["publishOn"])
			except Exception:
				pass
	return pd.DataFrame({"Stock":stock_list,"Content":content_list, "Title":title_list, "Date":date_list})



In [9]:
df = pd.DataFrame()

for s in stocks:
    temp = retrieve_earningscall(s)
    df = pd.concat([df, temp])

In [12]:
df = df[df["Date"] > "2019-12-01"] # keep only latest 12 quarters
df

,Stock,Content,Title,Date
0,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) Q4 2022 Earnings Call Transc...,2022-10-27T19:46:03-04:00
1,AAPL,"<p>Apple, Inc. (<span class=""ticker-hover-wrap...","Apple, Inc. (AAPL) CEO Tim Cook on Q3 2022 Res...",2022-07-28T21:11:05-04:00
2,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2022 Resu...,2022-04-28T19:21:03-04:00
3,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2022 Resu...,2022-01-27T20:19:05-05:00
4,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2021 Resu...,2021-10-28T21:25:05-04:00
5,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2021 Resu...,2021-07-27T23:13:05-04:00
6,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2021 Resu...,2021-04-28T21:17:11-04:00
7,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2021 Resu...,2021-01-27T20:48:03-05:00
8,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,2020-10-29T23:40:53-04:00
9,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,2020-07-30T23:35:04-04:00


In [13]:
import numpy as np
import pandas as pd
import datetime as dt
import re
import nltk
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [22]:
df['Conference'] = df["Content"].apply(lambda x: re.sub('<[^<]+?>', '', x.split("Question-and-Answer Session")[0]))
df['QnA'] = df["Content"].apply(lambda x: x.split("Question-and-Answer Session")[1] if len(x.split("Question-and-Answer Session")) >1  else None)
def f(row):
    qns_list = []
    if row["QnA"] != None:    
        for i in row["QnA"].split('<span class="question">'):
            foo = i.split('<span class="answer">')
            if len(foo[0]) > 450:
                s = re.sub('<[^<]+?>', '',foo[0])
                qns_list.append((' '.join(s.split()[2:])))
    else:
        return None
    if len(qns_list) == 0:
        return None
    return qns_list

df["Questions"] = df.apply(f, axis = 1)
df


,Stock,Content,Title,Date,Conference,QnA,Questions
0,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) Q4 2022 Earnings Call Transc...,2022-10-27T19:46:03-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2022 Earnings Conf...,"</i><i></i></strong></p> <p class=""paywall-fu...",[go ahead and take our first question from Sha...
1,AAPL,"<p>Apple, Inc. (<span class=""ticker-hover-wrap...","Apple, Inc. (AAPL) CEO Tim Cook on Q3 2022 Res...",2022-07-28T21:11:05-04:00,"Apple, Inc. (NASDAQ:AAPL) Q3 2022 Earnings Con...","</strong></p> <p class=""paywall-full-content"">...",[I guess two from my side. Maybe to start on t...
2,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2022 Resu...,2022-04-28T19:21:03-04:00,Apple Inc. (NASDAQ:AAPL) Q2 2022 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Yes. Thank you. Congrats on the quarter. A co...
3,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2022 Resu...,2022-01-27T20:19:05-05:00,Apple Inc. (NASDAQ:AAPL) Q1 2022 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Yes. Thank you. Your margins have clearly bee...
4,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2021 Resu...,2021-10-28T21:25:05-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...","[Thank you very much. Tim, I’m wondering, can ..."
5,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2021 Resu...,2021-07-27T23:13:05-04:00,Apple Inc. (NASDAQ:AAPL) Q3 2021 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Thank you. If I could follow-up with regard t...
6,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2021 Resu...,2021-04-28T21:17:11-04:00,Apple Inc. (NASDAQ:AAPL) Q2 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...","[Tim, I had sort of a big picture question on ..."
7,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2021 Resu...,2021-01-27T20:48:03-05:00,Apple Inc. (NASDAQ:AAPL) Q1 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...",[Congratulations on a really strong quarter. F...
8,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,2020-10-29T23:40:53-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...","[you very much. Tim, can you talk a bit more a..."
9,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,2020-07-30T23:35:04-04:00,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...",[Thank you. Good afternoon. Tim in light of th...


In [23]:
def question_sa(row): #Gets sentiment from the list of qna 
    labels = {0:'positive', 1:'negative'}
    sent_val = list()
    if row['Questions'] != None:
        for x in row['Questions']:
            try:
                inputs = tokenizer(x, return_tensors="pt", padding=True)
                outputs = finbert(**inputs)[0]
                val = labels[np.argmax(outputs.detach().numpy()[:,1:])]
                if val == "negative":
                    sent_val.append(-1)
                else:
                    sent_val.append(1)
            except Exception:
                pass
        if len(sent_val) != 0:
            return sum(sent_val)/len(sent_val)
        else:
            return None
    else:
        return None
    
df['Questions Sentiment'] = df.apply(question_sa, axis = 1)

In [24]:
def conference_sa(row): #Gets sentiment from the list of manager conference text
    labels = {0:'positive', 1:'negative'}
    sent_val = list()
    x = row['Conference']
    if len(x.split())//100 > 0:
        new_x = np.array_split(x.split(), len(x.split())//100)
    else:
        new_x = [x]
    for x in new_x:
        x = ' '.join(x)  
        try:
            inputs = tokenizer(x, return_tensors="pt", padding=True)
            outputs = finbert(**inputs)[0]
            val = labels[np.argmax(outputs.detach().numpy()[:,1:])]
            if val == "negative":
                sent_val.append(-1)
            else:
                sent_val.append(1)
        except Exception:
            pass
    if len(sent_val) != 0:
        return sum(sent_val)/len(sent_val)
    else:
        return None

df['Conference Sentiment'] = df.apply(conference_sa, axis = 1)
df["Has_QnA"] = np.where(df['Questions Sentiment'].isna(), 0, 1) # Checks whether QnA are available, some quarters might not have a qna section

In [25]:
df

,Stock,Content,Title,Date,Conference,QnA,Questions,Questions Sentiment,Conference Sentiment,Has_QnA
0,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) Q4 2022 Earnings Call Transc...,2022-10-27T19:46:03-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2022 Earnings Conf...,"</i><i></i></strong></p> <p class=""paywall-fu...",[go ahead and take our first question from Sha...,-0.500000,0.621622,1
1,AAPL,"<p>Apple, Inc. (<span class=""ticker-hover-wrap...","Apple, Inc. (AAPL) CEO Tim Cook on Q3 2022 Res...",2022-07-28T21:11:05-04:00,"Apple, Inc. (NASDAQ:AAPL) Q3 2022 Earnings Con...","</strong></p> <p class=""paywall-full-content"">...",[I guess two from my side. Maybe to start on t...,-0.875000,0.677419,1
2,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2022 Resu...,2022-04-28T19:21:03-04:00,Apple Inc. (NASDAQ:AAPL) Q2 2022 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Yes. Thank you. Congrats on the quarter. A co...,-0.466667,0.687500,1
3,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2022 Resu...,2022-01-27T20:19:05-05:00,Apple Inc. (NASDAQ:AAPL) Q1 2022 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Yes. Thank you. Your margins have clearly bee...,0.000000,0.575758,1
4,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2021 Resu...,2021-10-28T21:25:05-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...","[Thank you very much. Tim, I’m wondering, can ...",-0.666667,0.714286,1
5,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2021 Resu...,2021-07-27T23:13:05-04:00,Apple Inc. (NASDAQ:AAPL) Q3 2021 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...",[Thank you. If I could follow-up with regard t...,-0.333333,0.818182,1
6,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q2 2021 Resu...,2021-04-28T21:17:11-04:00,Apple Inc. (NASDAQ:AAPL) Q2 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...","[Tim, I had sort of a big picture question on ...",-0.600000,0.823529,1
7,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q1 2021 Resu...,2021-01-27T20:48:03-05:00,Apple Inc. (NASDAQ:AAPL) Q1 2021 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...",[Congratulations on a really strong quarter. F...,-0.066667,0.636364,1
8,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,2020-10-29T23:40:53-04:00,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"</strong></p> <p class=""paywall-full-content"">...","[you very much. Tim, can you talk a bit more a...",-0.230769,0.500000,1
9,AAPL,"<p>Apple Inc. (<span class=""ticker-hover-wrapp...",Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,2020-07-30T23:35:04-04:00,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"</strong></p> <p class=""paywall-full-content"">...",[Thank you. Good afternoon. Tim in light of th...,0.000000,0.567568,1


In [ ]:
df.to_csv("../data/datafiles/earnings_call.csv")